In [536]:
from tqdm import tqdm
import io
import numpy as np
import pandas as pd
import requests

In [537]:
def hubeau_get_stations_ades(numéro_département: int) -> pd.DataFrame:
    """Récupérer la liste (fichier CSV) des points d'eau d'un département et la retourner sous forme d'un DataFrame. Exemple de requête :
    
    https://hubeau.eaufrance.fr/api/v1/niveaux_nappes/stations.csv?code_departement=66&size=200
    """
    url = 'https://hubeau.eaufrance.fr/api/v1/niveaux_nappes/stations.csv'
    grand_nombre_magique = 9999
    query_params = {'code_departement': numéro_département, 'size': grand_nombre_magique}
    response = requests.get(url, params=query_params, )
    df_liste_stations = pd.read_csv(io.BytesIO(response.content), sep=';')
    return df_liste_stations

In [538]:
def get_chroniques_station(infos_station: dict) -> pd.DataFrame:
    """Récupérer les données d'intérêt sur le code BSS d'une station. Exemple de requête :
    
    https://hubeau.eaufrance.fr/api/v1/niveaux_nappes/chroniques.csv?code_bss=10908X0136/DUCUP2&size=20000
    """
    url = 'https://hubeau.eaufrance.fr/api/v1/niveaux_nappes/chroniques.csv'
    query_params = {
        'code_bss': infos_station.get('code_bss'), 'size': infos_station.get('nb_mesures_piezo')}
    response = requests.get(url, params=query_params)
    if response.headers.get('Content-Length') == 0 \
            or 'Content-Encoding' not in response.headers:
        return None
    df = pd.read_csv(io.BytesIO(response.content), sep=';')
    return df

In [539]:
def get_référentiel_IPS(liste_indicateurs_toute_la_période: list) -> dict:
    """Retourne un dictionnaire composé de couples {libellé: intervalle de valeurs IPS}, 
    autrement dit {"un libellé": [valeur inf., valeur sup.]} :
    
    {
    'Niveaux très bas':             [0, 0.14285714285714285],
    'Niveaux bas':                  [0.14295714285714284, 0.2857142857142857],
    'Niveaux modérément bas':       [0.2858142857142857, 0.42857142857142855],
    'Niveaux autour de la moyenne': [0.42867142857142854, 0.5714285714285714],
    'Niveaux modérément hauts':     [0.5715285714285714, 0.7142857142857142],
    'Niveaux hauts':                [0.7143857142857142, 0.8571428571428571],
    'Niveaux très hauts':           [0.8572428571428571, 1.0]
    }
    
    D'après la documentation :
    - https://ades.eaufrance.fr/Spip?p=IMG/pdf/index_piezo_final.pdf#page=2
    - http://infoterre.brgm.fr/rapports/RP-61807-FR.pdf#page=80
    """
    ips_min = min(liste_indicateurs_toute_la_période)
    ips_max = max(liste_indicateurs_toute_la_période)
    ips_valeurs = np.linspace(ips_min, ips_max, 7 + 1)
    ips_libellés = ['très bas', 'bas', 'modérément bas', 'autour de la moyenne', 'modérément hauts', 'hauts', 'très hauts']
    ips_référentiel = {}
    petit_décalage = 0
    for i, (lib, val) in enumerate(zip(ips_libellés, ips_valeurs[1:])):
        if i == 0:
            val_inf, val_sup = [0, val]
        else:
            val_inf, val_sup = [ips_valeurs[i] + petit_décalage, val]
        ips_référentiel |= {'Niveaux ' + lib: [val_inf, val_sup]}
    return ips_référentiel    

In [540]:
def déterminer_libellé_IPS(indicateur: float, ips_référentiel: dict) -> str:
    """Détermine le libellé pour un indicateur IPS donné en entrée.
    
    D'après la documentation :
    - https://ades.eaufrance.fr/Spip?p=IMG/pdf/index_piezo_final.pdf#page=2
    - http://infoterre.brgm.fr/rapports/RP-61807-FR.pdf#page=80
    """
    if np.isnan(indicateur):
        return ''
    for libellé, (val_inf, val_sup) in ips_référentiel.items():
        if indicateur == 0 and val_inf == 0:
            return libellé
        if val_inf < indicateur <= val_sup:
            return libellé
    raise ValueError(f'Valeur {indicateur} ne rentre dans aucun des intervalles de notre référentiel IPS')

In [541]:
def hubeau_get_chroniques_des_stations(df_liste_stations: pd.DataFrame, nom_fichier_destination: str) -> pd.DataFrame:
    """Récupérer, pour chaque station de la liste, les données d'intérêt et les sauvegarder itérativement dans un fichier CSV
    """
    infos_stations_liste = df_liste_stations.to_dict(orient='records')
    garder_lentête = True
    with open(nom_fichier_destination, 'w', newline='\n') as fichier_csv_cible:
        fichier_csv_cible.write('')
    with tqdm(total=len(infos_stations_liste), ncols=100) as progr_bar:
        for infos_station in infos_stations_liste[:]:
            progr_bar.set_description(f"Station {infos_station.get('code_bss')}")
            df_chroniques_station = get_chroniques_station(infos_station)
            progr_bar.update()
            if df_chroniques_station is None:
                continue
            df_tmp = pd.merge(left=pd.DataFrame([infos_station]),
                              right=df_chroniques_station,
                              on=['code_bss', 'urn_bss'], how='outer')
            niveau_min = df_tmp['niveau_nappe_eau'].min()
            niveau_max = df_tmp['niveau_nappe_eau'].max()
            calcul_IPS = lambda x: pd.NA if np.isnan(x['niveau_nappe_eau']) else (x['niveau_nappe_eau'] - niveau_min) / (niveau_max - niveau_min)
            df_tmp['indicateur_piezometrique_standardise'] = df_tmp.apply(calcul_IPS, axis=1)
            liste_indicateurs_toute_la_période = df_tmp['indicateur_piezometrique_standardise'].to_list()
            ips_référentiel = get_référentiel_IPS(liste_indicateurs_toute_la_période)
            df_tmp['libelle_indicateur_piezometrique_standardise'] = df_tmp['indicateur_piezometrique_standardise'].apply(lambda x: déterminer_libellé_IPS(x, ips_référentiel))
            df_tmp.to_csv(nom_fichier_destination, mode='a',
                          sep=';', index=False, header=garder_lentête)
            garder_lentête = False
    return pd.read_csv(nom_fichier_destination, sep=';', low_memory=False)

In [542]:
numéro_département = 66
nom_fichier_destination = f'hubeau_chroniques_des_stations_ades_dans_{numéro_département}.csv'

In [543]:
df_liste_stations = hubeau_get_stations_ades(numéro_département)
print(f"Il y a {len(df_liste_stations)} stations dans le département {numéro_département}.")
df_liste_stations_avec_mesures = df_liste_stations[df_liste_stations['nb_mesures_piezo'] > 0]
print(f"Il y a {len(df_liste_stations_avec_mesures)} stations pour lesquelles il existe au moins une mesure enregistrée.")
df_infos_stations_et_chroniques = hubeau_get_chroniques_des_stations(df_liste_stations_avec_mesures, nom_fichier_destination)

Il y a 86 stations dans le département 66.
Il y a 83 stations pour lesquelles il existe au moins une mesure enregistrée.


Station 10984X0003/AIGUAN:  66%|███████████████████████▊            | 55/83 [01:57<00:36,  1.32s/it]C:\Users\Administrateur\AppData\Local\Temp\ipykernel_12544\2618744094.py:20: RuntimeWarning: invalid value encountered in scalar divide
  calcul_IPS = lambda x: pd.NA if np.isnan(x['niveau_nappe_eau']) else (x['niveau_nappe_eau'] - niveau_min) / (niveau_max - niveau_min)
Station 10915X0395/PZ: 100%|████████████████████████████████████████| 83/83 [02:42<00:00,  1.95s/it]


In [544]:
df_infos_stations_et_chroniques = pd.read_csv(nom_fichier_destination, sep=';', low_memory=False)

In [545]:
colonnes_dans_lordre = ['code_bss', 'date_debut_mesure', 'date_fin_mesure',
                        'code_commune_insee', 'nom_commune', 'x', 'y', 'codes_bdlisa',
                        'bss_id', 'altitude_station', 'nb_mesures_piezo',
                        'code_departement', 'nom_departement', 'libelle_pe',
                        'profondeur_investigation', 'codes_masse_eau_edl', 'noms_masse_eau_edl',
                        'date_maj', 'date_mesure', 'timestamp_mesure',
                        'niveau_nappe_eau', 'mode_obtention', 'statut', 'qualification',
                        'code_continuite', 'nom_continuite', 'code_producteur',
                        'nom_producteur', 'code_nature_mesure', 'nom_nature_mesure',
                        'profondeur_nappe', 'indicateur_piezometrique_standardise', 
                        'libelle_indicateur_piezometrique_standardise',
                        'urn_bss', 'urns_bdlisa', 'urns_masse_eau_edl']
df_infos_stations_et_chroniques = df_infos_stations_et_chroniques[colonnes_dans_lordre]
#
colonnes_renommées = {'x': 'longitude_x', 'y': 'latitude_y'}
df_infos_stations_et_chroniques.rename(columns=colonnes_renommées, inplace=True)

In [550]:
df_infos_stations_et_chroniques['libelle_indicateur_piezometrique_standardise'].value_counts(dropna=False)

libelle_indicateur_piezometrique_standardise
Niveaux bas                     65325
Niveaux autour de la moyenne    60851
Niveaux très bas                58089
Niveaux modérément bas          55686
Niveaux modérément hauts        55024
Niveaux hauts                   45388
Niveaux très hauts              17540
NaN                               719
Name: count, dtype: int64

In [547]:
df_infos_stations_et_chroniques.to_csv(nom_fichier_destination, index=False, sep=';')

Valider le fichier CSV produit en comptant le nombre de champs de chaque ligne:

In [548]:
field_sep = ';'
header = ''
how_many_fields_in_header = 0
with open(nom_fichier_destination, 'r') as fichier_csv_cible:
    for i, line in enumerate(fichier_csv_cible, start=1):
        if i == 1:
            header = line
            how_many_fields_in_header = line.count(field_sep)
        if line.count(field_sep) != how_many_fields_in_header:
            print(str(i).zfill(10), ':', header)
            print(str(i).zfill(10), ':', line)
            break

Schéma types de données pour BigQuery :

In [549]:
for c in df_infos_stations_et_chroniques.columns:
    dtype = df_infos_stations_et_chroniques[c].dtype.name.replace('object', 'string')
    print(f"{c}:{dtype}", end=',\n')

code_bss:string,
date_debut_mesure:string,
date_fin_mesure:string,
code_commune_insee:int64,
nom_commune:string,
longitude_x:float64,
latitude_y:float64,
codes_bdlisa:string,
bss_id:string,
altitude_station:float64,
nb_mesures_piezo:int64,
code_departement:int64,
nom_departement:string,
libelle_pe:string,
profondeur_investigation:float64,
codes_masse_eau_edl:string,
noms_masse_eau_edl:string,
date_maj:string,
date_mesure:string,
timestamp_mesure:int64,
niveau_nappe_eau:float64,
mode_obtention:string,
statut:string,
qualification:string,
code_continuite:float64,
nom_continuite:string,
code_producteur:int64,
nom_producteur:string,
code_nature_mesure:string,
nom_nature_mesure:string,
profondeur_nappe:float64,
indicateur_piezometrique_standardise:float64,
libelle_indicateur_piezometrique_standardise:string,
urn_bss:string,
urns_bdlisa:string,
urns_masse_eau_edl:string,
